# Estimering av Sykehusopphold

Maskinlæringsmodellen har som mål å predikere den forventede lengden på sykehusoppholdet per pasient. I tillegg skal lengden på sykehusopphold være basert på passende variabler fra data på pasientopplysninger, inkludert fysiologiske, demografiske og sykdomsalvorlighetsdata på tvers av ni sykdomskategorier.

### Importere nødvendinge datapakker

In [1924]:
import numpy as np 
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import pickle

import scipy.stats as stats
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### Les inn data

In [1925]:
df_demographic = pd.read_csv("raw_data/demographic.csv")
df_hospital = pd.read_csv("raw_data/hospital.csv")
df_physiological = pd.read_csv("raw_data/physiological.txt", sep="\t")
df_severity = pd.read_json("raw_data/severity.json")

## Preprocessing

### Rydde data

Lager en hjelpefunksjoner som skal rydde skal rydde de ulike datasettene

In [1926]:
def clean_demographic_data(df_demographic):
    non_negative_cat = ["alder", "utdanning"]
    
    for col in non_negative_cat:
        df_demographic.loc[df_demographic[col] < 0, col] = np.nan # Bytter alle negative verdier (som ikke skal være negative) til NaN
    
    df_demographic = df_demographic.drop_duplicates() # Fjerner alle duplikater
    
    return df_demographic

In [1927]:
def clean_hospital_data(df_hospital, is_raw = True):
    
    df_hospital = df_hospital.drop(["sykehusdød"], axis=1) # Ikke hensiktsmessig å ta med
    
    # Kun raw_data inneholder "oppholdslengde"
    if is_raw:
        non_negative_cat = ["oppholdslengde"]
        for col in non_negative_cat:
            df_hospital.loc[df_hospital[col] < 0, col] = np.nan # Bytter alle negative verdier (som ikke skal være negative) til NaN
    
    df_hospital = df_hospital.drop_duplicates()
    
    return df_hospital

In [1928]:
def clean_severity_data(df):
    severity_var_list = df.columns.tolist()
    df = df.explode(severity_var_list[2:]) # Fra og med "pasient_id" til og med siste kolonne
    # Sorterer dataframen slik at pasient_id er først
    temp_cols = ["pasient_id"]
    for col in df.columns:
        if col != "pasient_id":
            temp_cols.append(col)
    df = df[temp_cols]
    
    # Fjerner sykdomskategori_id og sykdomskategori, da sykdom_underkategori forteller oss det samme bare mer detaljert
    df = df.drop(["sykdomskategori_id", "sykdomskategori"], axis=1) 
    
    df = df.drop(["adl_pasient"], axis=1) # adl_pasient er fylt inn på dag 7
    df = df.drop(["dødsfall"], axis=1) # Ikke hensiktsmessig å ta med
    
    non_negative_cat = ["antall_komorbiditeter", "koma_score", "fysiologisk_score", 
                        "apache_fysiologisk_score", "overlevelsesestimat_2mnd", "overlevelsesestimat_6mnd",
                        "diabetes", "demens", "lege_overlevelsesestimat_2mnd", "lege_overlevelsesestimat_6mnd"]
    
    for col in non_negative_cat:
        df.loc[df[col] < 0, col] = np.nan # Bytter alle negative verdier (som ikke skal være negative) til NaN
    
    numeric_cols = ["antall_komorbiditeter", "koma_score", "adl_stedfortreder","fysiologisk_score", 
                    "apache_fysiologisk_score", "overlevelsesestimat_2mnd", "overlevelsesestimat_6mnd",
                    "lege_overlevelsesestimat_2mnd", "lege_overlevelsesestimat_6mnd"]
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    df = df.drop_duplicates()
    df = df.sort_values(by="pasient_id")
    df = df.reset_index(drop=True)
    return df

In [1929]:
def clean_physiological_data(df_physiological):
    df_physiological = df_physiological.drop(["bilirubin"], axis=1) # bilirubin fylt inn på dag 7
    
    non_negative_cat = df_physiological.columns
    
    for col in non_negative_cat:
        df_physiological.loc[df_physiological[col] < 0, col] = np.nan # Bytter alle negative verdier (som ikke skal være negative) til NaN
    
    df_physiological = df_physiological.drop_duplicates()
    
    return df_physiological

In [1930]:
df_demographic = clean_demographic_data(df_demographic)
df_hospital = clean_hospital_data(df_hospital)
df_severity = clean_severity_data(df_severity)
df_physiological = clean_physiological_data(df_physiological)

Deretter settes alle dataframene til et enkelt dataframe

In [1931]:
def merge_dataframes(df_hospital, df_demographic, df_physiological, df_severity):
    df = pd.merge(df_hospital, df_demographic, "outer", "pasient_id")
    df = pd.merge(df, df_physiological, "outer", "pasient_id")
    df = pd.merge(df, df_severity, "outer", "pasient_id")
    assert df["pasient_id"].duplicated().any() == False, "Det er duplikater av pasient_id"
    return df

In [1932]:
df = merge_dataframes(df_hospital, df_demographic, df_physiological, df_severity)
df["pasient_id"].duplicated().any() == False, "Det er duplikater av pasient_id"
df.head()

,pasient_id,oppholdslengde,alder,kjønn,utdanning,inntekt,etnisitet,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde,sykdom_underkategori,antall_komorbiditeter,koma_score,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,dnr_dag
0,2,4.0,60.33899,female,12.0,$11-$25k,white,43.0,17.097656,112.0,34.0,34.59375,98.00000,NaN,5.500000,132.0,7.250000,NaN,NaN,NaN,Cirrhosis,2,44.0,1.0,52.695312,74.0,0.001000,0.000000,0,0,no,0.00,0.0,None,None
1,3,17.0,52.74698,female,12.0,under $11k,white,70.0,8.500000,88.0,28.0,37.39844,231.65625,NaN,2.000000,134.0,7.459961,NaN,NaN,NaN,Cirrhosis,2,0.0,0.0,20.500000,45.0,0.790894,0.664917,0,0,no,0.75,0.5,None,None
2,4,3.0,42.38498,female,11.0,under $11k,white,75.0,9.099609,88.0,32.0,35.00000,NaN,NaN,0.799927,139.0,NaN,NaN,NaN,NaN,Lung Cancer,2,0.0,0.0,20.097656,19.0,0.698975,0.411987,0,0,metastatic,0.90,0.5,None,None
3,5,NaN,79.88495,female,NaN,NaN,white,59.0,13.500000,112.0,20.0,37.89844,173.31250,NaN,0.799927,143.0,7.509766,NaN,NaN,NaN,ARF/MOSF w/Sepsis,1,26.0,2.0,23.500000,30.0,0.634888,0.532959,0,0,no,0.90,0.9,None,None
4,6,4.0,93.01599,male,14.0,NaN,white,110.0,10.398438,101.0,44.0,38.39844,266.62500,NaN,0.699951,140.0,7.659180,NaN,NaN,NaN,Coma,1,55.0,1.0,19.398438,27.0,0.284973,0.214996,0,0,no,0.00,0.0,None,None


Da oppholdslengden er målvariabelen, må man fjerne alle rader hvor det mangler oppholdslengde. I tillegg fjernes pasient_id, da dette ikke vil ha noe direkte innvirkning på sykehusopphold.

In [1933]:
df = df.dropna(subset=["oppholdslengde"], axis=0) # Fjerner alle rader hvor oppholdslengde mangler, da det er variabelen modellen skal predikere
df = df.drop(["pasient_id"], axis=1)

Lager en hjelpefunksjon slik at man slepper å gjøre alt på ny når man skal senere importere sample_data

In [1934]:
def prepare_data(df_hospital, df_demographic, df_physiological, df_severity):
    
    df_hospital = clean_hospital_data(df_hospital, is_raw=False)
    df_demographic = clean_demographic_data(df_demographic)
    df_severity = clean_severity_data(df_severity)
    df_physiological = clean_physiological_data(df_physiological)
    
    df = merge_dataframes(df_hospital, df_demographic, df_physiological, df_severity)
    
    return df 

### Train_test_split
Før man begynner med datamodellering må man splitte data i trenings-, valederings- og testdata

In [1935]:
X = df.drop("oppholdslengde", axis=1)
y = df["oppholdslengde"]

X_train, X_temp, y_train, y_temp = train_test_split(X, y , train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp , test_size=0.5, random_state=42)

### Utforskende dataanalyse, manglende data og feature extraction (variabelutvinning)

#### Tetthetsfunksjoner

Bruker innebygde funksjoner i pandas for å få mer info om X_train.

In [1936]:
X_train.describe()

,alder,utdanning,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde,antall_komorbiditeter,koma_score,adl_stedfortreder,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd
count,5408.000000,4435.000000,5413.000000,5281.000000,5413.000000,5413.000000,5413.000000,4040.000000,3421.000000,5371.000000,5413.000000,4059.000000,2753.000000,2845.000000,2549.000000,5413.000000,5413.000000,3699.000000,5413.000000,5413.000000,5413.000000,5413.000000,4429.000000,4441.000000
mean,62.725937,11.802480,84.210881,12.411832,97.694624,23.482912,37.145275,237.822606,2.941227,1.764292,137.586181,7.415445,159.633491,32.083585,2194.752485,1.855163,12.440421,1.625574,25.527389,37.635138,0.632466,0.517226,0.615295,0.494693
std,15.637994,3.421883,27.700134,8.989788,31.487143,9.639604,1.264891,109.478257,0.924940,1.652531,6.014258,0.081055,87.051063,26.086755,1454.466754,1.336317,25.085523,2.223967,9.940611,19.975809,0.250956,0.255474,0.300149,0.303345
min,18.118990,0.000000,0.000000,0.000000,0.000000,0.000000,31.699220,12.000000,0.399963,0.099991,110.000000,6.829102,1.399902,1.000000,0.000000,0.000000,0.000000,0.000000,1.199951,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.863470,10.000000,63.000000,7.000000,72.000000,18.000000,36.195310,154.468750,2.399902,0.899902,134.000000,7.379883,103.000000,14.000000,1180.000000,1.000000,0.000000,0.000000,19.000000,23.000000,0.501953,0.333984,0.400000,0.200000
50%,65.015990,12.000000,77.000000,10.699219,100.000000,24.000000,36.796880,222.500000,2.899902,1.199951,137.000000,7.429688,135.000000,23.000000,1963.000000,2.000000,0.000000,0.000000,23.898438,35.000000,0.711914,0.570923,0.700000,0.500000
75%,74.116718,14.000000,107.000000,15.398438,120.000000,28.000000,38.195310,304.000000,3.599609,1.899902,141.000000,7.469727,189.000000,42.000000,2965.000000,3.000000,9.000000,3.000000,30.199219,49.000000,0.823975,0.724976,0.900000,0.750000
max,100.849000,31.000000,195.000000,113.593750,232.000000,90.000000,41.695310,869.375000,29.000000,18.398438,181.000000,7.709961,1092.000000,171.000000,9000.000000,9.000000,100.000000,7.000000,98.000000,143.000000,0.966919,0.943970,1.000000,1.000000


La oss se på noen interessante variabler.

In [1937]:
cols_to_watch = ["alder", "blodtrykk", "hjertefrekvens", "fysiologisk_score"]

for col in cols_to_watch:
    mu, std, N = np.mean(X_train[col]), np.std(X_train[col]), 1001
    x = np.linspace(mu - 3*std, mu + 3*std, N)
    fx = stats.norm.pdf(x, mu, std)
    dmax = max(fx) + 0.02
    
    hist = px.histogram(X_train, x=col, histnorm="probability density")
    fig = go.Figure(data=hist.data)
    fig.add_trace(go.Scatter(x=x, y=fx, mode="lines", line=dict(color="red"), name="Normalfordeling"))
    fig.add_trace(go.Scatter(x=[mu, mu], y=[0, dmax], mode="lines", line=dict(color="yellow"),name="Middelverdi"))
    fig.add_trace(go.Scatter(x=[np.median(X_train[col]), np.median(X_train[col])], y=[0, dmax], mode="lines", line=dict(color="purple"),name="Median"))
    fig.add_trace(go.Scatter(x=[stats.mode(X_train[col])[0], stats.mode(X_train[col])[0]], y=[0, dmax], mode="lines", line=dict(color="orange"),name="Modus"))
    fig.update_xaxes(title_text=col)
    fig.update_yaxes(title_text="probability density")
    fig.write_image(f"images/{col}_tetthetsfunksjon.png")
    fig.show()

Sjekker også fordelingen for målvariabelen, oppholdslengde.

In [1938]:
mu, std, N = np.mean(y_train), np.std(y_train), 100
x = np.linspace(mu - 3*std, mu + 3*std, N)
fx = stats.norm.pdf(x, mu, std)
dmax = max(fx) + 0.02

hist = px.histogram(x=y_train, histnorm="probability density")
fig = go.Figure(data=hist.data)
fig.add_trace(go.Scatter(x=x, y=fx, mode="lines", line=dict(color="red"), name="Normalfordeling"))
fig.add_trace(go.Scatter(x=[mu, mu], y=[0, dmax], mode="lines", line=dict(color="yellow"),name="Middelverdi"))
fig.add_trace(go.Scatter(x=[np.median(y_train), np.median(y_train)], y=[0, dmax], mode="lines", line=dict(color="purple"),name="Median"))
fig.add_trace(go.Scatter(x=[stats.mode(y_train)[0], stats.mode(y_train)[0]], y=[0, dmax], mode="lines", line=dict(color="orange"),name="Modus"))
fig.update_xaxes(title_text="oppholdslengde")
fig.update_yaxes(title_text="probability density")
fig.write_image(f"images/oppholdslengde_tetthetsfunksjon.png")
fig.show()

#### Manglende data

In [1939]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5413 entries, 6970 to 7276
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   alder                          5408 non-null   float64
 1   kjønn                          5413 non-null   object 
 2   utdanning                      4435 non-null   float64
 3   inntekt                        3620 non-null   object 
 4   etnisitet                      5382 non-null   object 
 5   blodtrykk                      5413 non-null   float64
 6   hvite_blodlegemer              5281 non-null   float64
 7   hjertefrekvens                 5413 non-null   float64
 8   respirasjonsfrekvens           5413 non-null   float64
 9   kroppstemperatur               5413 non-null   float64
 10  lungefunksjon                  4040 non-null   float64
 11  serumalbumin                   3421 non-null   float64
 12  kreatinin                      5371 non-null   flo

In [1940]:
missing_values = X_train.isna().sum().sort_values(ascending=False)
missing_df = missing_values.to_frame().rename(columns={0: "Antall manglende verdier"})
missing_df = missing_df[missing_df["Antall manglende verdier"] > 0] # Vil kun se på dem som mangler verdier
missing_df["Prosent manglende verdier"] = (missing_df["Antall manglende verdier"] / X_train.shape[0]) * 100 # X_train.shape[0] er antall rader
missing_df

,Antall manglende verdier,Prosent manglende verdier
dnr_dag,4657,86.033623
dnr_status,4657,86.033623
urinmengde,2864,52.909662
glukose,2660,49.140957
blodurea_nitrogen,2568,47.441345
serumalbumin,1992,36.800296
inntekt,1793,33.123961
adl_stedfortreder,1714,31.664511
lungefunksjon,1373,25.364862
blod_ph,1354,25.013856


Ut fra resultatet velger jeg å fjerne alle som mangler mer enn 30% av data.

In [1941]:
df_to_drop = missing_df.loc[missing_df["Prosent manglende verdier"] >= 30, :]
cols_to_drop = df_to_drop.index.to_list()
cols_to_drop

['dnr_dag',
 'dnr_status',
 'urinmengde',
 'glukose',
 'blodurea_nitrogen',
 'serumalbumin',
 'inntekt',
 'adl_stedfortreder']

In [1942]:
def remove_cols(X_org, cols_to_drop=cols_to_drop):
    X = X_org.copy()
    X = X.drop(cols_to_drop, axis=1)
    return X

X_train = remove_cols(X_train)
X_val = remove_cols(X_val)
X_test = remove_cols(X_test)

Fjerner dem også fra missing_df.

In [1943]:
missing_df = missing_df.drop(cols_to_drop, axis=0)

#### Ulike imputeringsstrategier
La oss se effekten av ulike impteringsstrategier på de to kolonnene med mest manglende data etter man har fjernet dem over terskelen.

In [1954]:
missing_cols = missing_df.index.to_list() # Kolonner som inneholder manglende data
cols = missing_cols[0:2]
imputer_strats = [
    SimpleImputer(strategy="mean"),
    SimpleImputer(strategy="median"),
    SimpleImputer(strategy="most_frequent"),
    KNNImputer(),
    KNNImputer(n_neighbors=7)
]

titles = [f"{cols[0]}: Før imputering", f"{cols[1]}: Før imputering"]

for i in range(len(imputer_strats)):
    titles.append(f"{cols[0]}: {imputer_strats[i]}")
    titles.append(f"{cols[1]}: {imputer_strats[i]}")

fig = make_subplots(rows=1 + len(imputer_strats), cols=2, subplot_titles=titles)

for i, col in enumerate(cols):
    box = px.box(X_train, x=col)
    for trace in box.data:
        fig.add_trace(trace, row=1, col=i + 1)
    
    fig.update_xaxes(title_text="Value", row=1, col=i + 1)
    
    for j, imputer in enumerate(imputer_strats):
        try:
            X_train_imputed = X_train.copy()
            X_train_imputed[[col]] = imputer.fit_transform(X_train_imputed[[col]])
            
            box_imputed = px.box(X_train_imputed, x=col)
            for trace in box_imputed.data:
                fig.add_trace(trace, row=j + 2, col=i + 1)
        except Exception as error:
            print(error)
            continue
        
        fig.update_xaxes(title_text="Value", row=j + 2, col=i + 1)

fig.update_layout(height=1200, width=1000, title_text="Kolonner med manglende data og imputerte verdier")
fig.write_image(f"images/effekt_imputering.png")
fig.show()

Ut fra figuren ser man at ulike imputeringsstrategier kan gi ulike resultat. Senere skal det testes hvilke kombinasjoner av imputeringsstrategi og regresjonsmodell som fungerer best.

#### NEWS (National Early Warning Score)

NEWS er skåringssystem for målinger av vitale funksjoner hos syke personer. Høy NEWS score tilsier at alvorligheten av sykdommen er høy.

Kilde: https://sml.snl.no/NEWS_-_National_Early_Warning_Score

In [1955]:
def calculate_news_score(row):
    score = 0
    
    # Respirasjonsfrekvens 
    if row["respirasjonsfrekvens"] <= 8:
        score += 3
    elif row["respirasjonsfrekvens"] <= 11:
        score += 1
    elif row["respirasjonsfrekvens"] <= 20:
        score += 0
    elif row["respirasjonsfrekvens"] <= 24:
        score += 2
    else:
        score += 3
    
    # Blodtrykk
    if row["blodtrykk"] <= 90:
        score += 3
    elif row["blodtrykk"] <= 100:
        score += 2
    elif row["blodtrykk"] <= 110:
        score += 1
    elif row["blodtrykk"] <= 219:
        score += 0
    else:
        score += 3
    
    # Hjertefrekvens
    if row["hjertefrekvens"] <= 40:
        score += 3
    elif row["hjertefrekvens"] <= 50:
        score += 1
    elif row["hjertefrekvens"] <= 90:
        score += 0
    elif row["hjertefrekvens"] <= 110:
        score += 1
    elif row["hjertefrekvens"] <= 130:
        score += 2
    else:
        score += 3
    
    # Kroppstemperatur
    if row["kroppstemperatur"] <= 35.0:
        score += 3
    elif row["kroppstemperatur"] <= 36.0:
        score += 1
    elif row["kroppstemperatur"] <= 38.0:
        score += 0
    elif row["kroppstemperatur"] <= 39.0:
        score += 1
    else:
        score += 2
    
    return score

In [1956]:
def apply_news_score(X_org):
    X = X_org.copy()
    X["NEWS_score"] = X.apply(calculate_news_score, axis=1)
    return X

#### Dele inn i "normale" verdier
Vil sjekke om modellen får bedre effekt av å se på "normale" verdier for ulike variabler. I praksis blir det å gjøre numeriske variabler om til kategoriske.

Kilder:
https://litfl.com/pao2-fio2-ratio/

In [1957]:
def categorize_values(X_org):
    X = X_org.copy()
    X.loc[:, "respirasjonsfrekvens_range"] = pd.cut(X["respirasjonsfrekvens"], [0, 9, 12, 21, 25, np.inf], right=False)
    X.loc[:, "blodtrykk_range"] = pd.cut(X["blodtrykk"], [0, 91, 101, 111, 140, 160, 180, np.inf], right=False)
    X.loc[:, "lungefunksjon_range"] = pd.cut(X["lungefunksjon"], [0, 100, 201, 300, np.inf], right=False)
    X = X.drop(["blodtrykk", "respirasjonsfrekvens", "lungefunksjon"], axis=1)
    return X

Disse stegene implementeres i en hjelpefunksjon, transform_X.

In [1958]:
def transform_X(X_org):
    X = X_org.copy()
    X = apply_news_score(X)
    X = categorize_values(X)
    return X

def transform_Xs(X_train, X_val, X_test):
    return (transform_X(X_train), transform_X(X_val), transform_X(X_test))

In [1959]:
X_train_transformed, X_val_transformed, X_test_transformed = transform_Xs(X_train, X_val, X_test )

#### Kategoriske variabler --> numeriske variabler

Når man skal trene modeller må man gjøre kategoriske variabler om til numeriske.

In [1960]:
categorical_cols = ["sykdom_underkategori", "kreft", "kjønn", "etnisitet", "diabetes", "demens", "respirasjonsfrekvens_range", 
                    "blodtrykk_range", "lungefunksjon_range", "NEWS_score"
                    ]
numeric_cols = [col for col in X_train_transformed.columns if col not in categorical_cols] # Alle kolonner i X_train som ikke er kategoriske

encoder = ColumnTransformer([("cat", Pipeline([("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))]), categorical_cols), # Kategoriske variabler
                                 ("num", "passthrough", numeric_cols)], # Numeriske variabler forblir uendret
                            )

encoder.fit(X_train_transformed)

X_train_transformed_encoded = pd.DataFrame(encoder.transform(X_train_transformed), columns=encoder.get_feature_names_out())
X_val_transformed_encoded = pd.DataFrame(encoder.transform(X_val_transformed), columns=encoder.get_feature_names_out())
X_test_transformed_encoded = pd.DataFrame(encoder.transform(X_test_transformed), columns=encoder.get_feature_names_out())

assert X_train_transformed_encoded.columns.equals(X_val_transformed_encoded.columns) and X_val_transformed_encoded.columns.equals(X_test_transformed_encoded.columns), "Det er ikke like kolonner i trenings-, valderings og testdata"

In [1961]:
encoded_cols = X_train_transformed_encoded.columns
num_news_cols = len(encoded_cols) - len(X_train_transformed.columns)
print(f"Etter OneHotEncoder har man {num_news_cols} flere kolonner")

Etter OneHotEncoder har man 43 flere kolonner


#### Korrelasjon

Korrelasjon sier noe om linær sammenheng mellom to variabler. Hvis to variabler er korrelert, vil det foreslå at ved endring i den ene variabelen, vil også den andre variabelen endre seg. Man kan ha både positiv og negativ korrelasjon.

La oss først se på korrelasjonen mellom ulike variabler og målvariabelen, oppholdslengde.

In [1962]:
corr_oppholdslengde = X_train_transformed_encoded.corrwith(y_train)
corr_oppholdslengde = corr_oppholdslengde.sort_values()

corr_oppholdslengde_df = corr_oppholdslengde.reset_index()
corr_oppholdslengde_df.columns = ["Variabel", "Korrelasjon med oppholdslengde"]

fig = px.bar(corr_oppholdslengde_df, 
             x="Korrelasjon med oppholdslengde", 
             y="Variabel",
             title="Korrelasjon mellom ulike variabler og oppholdslengde",
             color="Korrelasjon med oppholdslengde"
             )
fig.write_image(f"images/korrelasjon.png")
fig.show()

Ut fra figuren ser man at absolutt korrelasjonsverdi er relativ lavt.

La oss heller se på korrelasjoner mellom ulike variabler.

In [1963]:
corr_mat = X_train_transformed_encoded.corr()
corr_df = corr_mat.unstack().reset_index()
corr_df.columns = ["Kolonne1", "Kolonne2", "Korrelasjon"]
corr_df = corr_df[corr_df['Kolonne1'] != corr_df['Kolonne2']]
corr_df = corr_df.sort_values(by="Korrelasjon")
corr_df = corr_df.iloc[::2] # Tar annenhver kolonne for å unngå dobbel reprentasjon av samme kombinasjon av to kolonner
corr_df

,Kolonne1,Kolonne2,Korrelasjon
771,cat__kjønn_female,cat__kjønn_male,-1.000000
1539,cat__demens_1,cat__demens_0,-1.000000
1399,cat__diabetes_1,cat__diabetes_0,-1.000000
983,cat__etnisitet_black,cat__etnisitet_white,-0.823050
4548,num__overlevelsesestimat_2mnd,num__fysiologisk_score,-0.753200
...,...,...,...
4552,num__overlevelsesestimat_2mnd,num__lege_overlevelsesestimat_2mnd,0.580168
422,cat__sykdom_underkategori_Lung Cancer,cat__kreft_metastatic,0.665694
4411,num__fysiologisk_score,num__apache_fysiologisk_score,0.798670
4691,num__lege_overlevelsesestimat_2mnd,num__lege_overlevelsesestimat_6mnd,0.897192


La oss se på variablene med absolutt korrelasjonsverdi høyere enn 0,7.

In [1964]:
high_corr_df = corr_df[abs(corr_df["Korrelasjon"]) > 0.7]
high_corr_df

,Kolonne1,Kolonne2,Korrelasjon
771,cat__kjønn_female,cat__kjønn_male,-1.000000
1539,cat__demens_1,cat__demens_0,-1.000000
1399,cat__diabetes_1,cat__diabetes_0,-1.000000
983,cat__etnisitet_black,cat__etnisitet_white,-0.823050
4548,num__overlevelsesestimat_2mnd,num__fysiologisk_score,-0.753200
561,cat__kreft_metastatic,cat__kreft_no,-0.703161
4411,num__fysiologisk_score,num__apache_fysiologisk_score,0.798670
4691,num__lege_overlevelsesestimat_2mnd,num__lege_overlevelsesestimat_6mnd,0.897192
4551,num__overlevelsesestimat_2mnd,num__overlevelsesestimat_6mnd,0.960550


Ut fra resulatet vil jeg droppe den ene kolonnen hvor absolutt korrelasjon er på 1. I tillegg vil jeg droppe noen kolonner som har høy korrelasjon, men lager nye "interaction" kolonner ut fra disse.

In [1965]:
def high_corr_interaction(X_encoded_org):
    X_encoded = X_encoded_org.copy()
    X_encoded["num__apache_fysiologisk_score_fysiologisk_score_interaction"] = X_encoded["num__apache_fysiologisk_score"] * X_encoded["num__fysiologisk_score"]
    X_encoded["num__lege_overlevelsesestimat_2mnd_lege_overlevelsesestimat_6mnd_interaction"] = X_encoded["num__lege_overlevelsesestimat_2mnd"] * X_encoded["num__lege_overlevelsesestimat_6mnd"]
    X_encoded["num__overlevelsesestimat_2mnd_overlevelsesestimat_6mnd_interaction"] = X_encoded["num__overlevelsesestimat_2mnd"] * X_encoded["num__overlevelsesestimat_6mnd"]
    X_encoded = X_encoded.drop(["num__apache_fysiologisk_score", "num__fysiologisk_score", 
                                "num__lege_overlevelsesestimat_2mnd", "num__lege_overlevelsesestimat_6mnd",
                                "num__overlevelsesestimat_2mnd", "num__overlevelsesestimat_6mnd"
                                ], axis=1)
    return X_encoded

def drop_high_corr(X_encoded_org):
    X_encoded = X_encoded_org.copy()
    X_encoded = X_encoded.drop(["cat__kjønn_female", "cat__demens_0", "cat__diabetes_0"], axis=1)
    X_encoded= high_corr_interaction(X_encoded)
    return X_encoded

def drop_high_corr_Xs(X_train, X_val, X_test):
    return drop_high_corr(X_train), drop_high_corr(X_val), drop_high_corr(X_test)

In [1966]:
X_train_final, X_val_final, X_test_final = drop_high_corr_Xs(X_train_transformed_encoded, X_val_transformed_encoded, X_test_transformed_encoded)

In [1967]:
X_train_final

,cat__sykdom_underkategori_ARF/MOSF w/Sepsis,cat__sykdom_underkategori_CHF,cat__sykdom_underkategori_COPD,cat__sykdom_underkategori_Cirrhosis,cat__sykdom_underkategori_Colon Cancer,cat__sykdom_underkategori_Coma,cat__sykdom_underkategori_Lung Cancer,cat__sykdom_underkategori_MOSF w/Malig,cat__kreft_metastatic,cat__kreft_no,cat__kreft_yes,cat__kjønn_male,cat__etnisitet_asian,cat__etnisitet_black,cat__etnisitet_hispanic,cat__etnisitet_other,cat__etnisitet_white,cat__etnisitet_nan,cat__diabetes_1,cat__demens_1,"cat__respirasjonsfrekvens_range_[0.0, 9.0)","cat__respirasjonsfrekvens_range_[9.0, 12.0)","cat__respirasjonsfrekvens_range_[12.0, 21.0)","cat__respirasjonsfrekvens_range_[21.0, 25.0)","cat__respirasjonsfrekvens_range_[25.0, inf)","cat__blodtrykk_range_[0.0, 91.0)","cat__blodtrykk_range_[91.0, 101.0)","cat__blodtrykk_range_[101.0, 111.0)","cat__blodtrykk_range_[111.0, 140.0)","cat__blodtrykk_range_[140.0, 160.0)","cat__blodtrykk_range_[160.0, 180.0)","cat__blodtrykk_range_[180.0, inf)","cat__lungefunksjon_range_[0.0, 100.0)","cat__lungefunksjon_range_[100.0, 201.0)","cat__lungefunksjon_range_[201.0, 300.0)","cat__lungefunksjon_range_[300.0, inf)",cat__lungefunksjon_range_nan,cat__NEWS_score_0,cat__NEWS_score_1,cat__NEWS_score_2,cat__NEWS_score_3,cat__NEWS_score_4,cat__NEWS_score_5,cat__NEWS_score_6,cat__NEWS_score_7,cat__NEWS_score_8,cat__NEWS_score_9,cat__NEWS_score_10,cat__NEWS_score_11,cat__NEWS_score_12,num__alder,num__utdanning,num__hvite_blodlegemer,num__hjertefrekvens,num__kroppstemperatur,num__kreatinin,num__natrium,num__blod_ph,num__antall_komorbiditeter,num__koma_score,num__apache_fysiologisk_score_fysiologisk_score_interaction,num__lege_overlevelsesestimat_2mnd_lege_overlevelsesestimat_6mnd_interaction,num__overlevelsesestimat_2mnd_overlevelsesestimat_6mnd_interaction
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.12500,12.0,8.898438,60.0,36.29688,0.899902,139.0,7.279297,3.0,0.0,358.660155,0.8100,0.425771
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,65.26495,12.0,18.500000,160.0,39.00000,1.500000,131.0,7.349609,5.0,44.0,3329.789062,0.0001,0.000235
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.98395,12.0,10.500000,74.0,38.09375,1.699951,137.0,7.429688,3.0,0.0,530.359375,0.8100,0.491157
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.26599,20.0,10.398438,60.0,35.59375,1.699951,135.0,NaN,1.0,0.0,818.296875,NaN,0.043329
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,69.77100,12.0,16.898438,165.0,38.19531,1.299805,138.0,7.369141,1.0,0.0,50.396485,NaN,0.606980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.71597,9.0,11.398438,76.0,35.79688,1.199951,137.0,7.359375,7.0,0.0,154.679688,0.5600,0.831993
5409,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

## Datamodellering

Målet er å trene ulike modeller på treningsdata og teste dem på valideringsdata. Den med minst feil på valederingsdata bestemmes som "beste" model. Til slutt ser man på generaliseringsevnen til den beste modellen ved å teste den på testdata. 

### Grunnlinjemodell

Aller først, lager man en grunnlinje modell for se hvor dårlig en modell vil fungere. I likhet som andre modeller må den først trenes vha. fit-metoden og deretter bruker man predict-metoden for å predikere y på nye data av X. I dette tilfelle med DummyRegressor(), predikerer vi gj.snittsverdien av y_val, uavhengig av hva X_val er.

In [1968]:
baseline = DummyRegressor(strategy="mean") # Lage modell
baseline.fit(X_train, y_train) # Tilpasse modell
prediction = baseline.predict(X_val) # Prediksjon på valederingsdata
rmse_baseline = root_mean_squared_error(y_val, prediction) # Sjekker RMSE mellom de faktiske dataene og prediksjonen
rmse_baseline 

20.603790732142762

Målet nå er å trene ulike modeller med testing av flere, ulike hyperparametre. For å teste hyperparametre kan man bruke GridSearchCV ved hjelp av en "parameter grid" som har en innebygd fit- og score-metode. Parameter grid er en grid med ulike parametre som skal testes. I dette tilfelle bruker jeg RandomizedSearchCV for å minimere kjøretid. Prinsippet er ganske likt, bortsett fra at RandomizedSearchCV ikke sjekker alle mulige kombinasjoner, men heller n kombinasjoner.

Kilde: https://youtu.be/0B5eIE_1vpU?si=EwWVMFx0aYt_b4e6

Først la oss kombinere alle preprossescing steps inn i pipeline for reproduserbarhet. Da blir det lettere å reprodusere metodene når man skal senere implentere beste modellen for en nettside. Da man skal teste flere ulike regresjonsmodeller samt flere preprocessingsstrategier, trenger man aller først en placeholder.

#### Placeholder pipieline

In [1969]:
pipeline = Pipeline([
    ("preprocessing", None), # Placeholder
    ("model", None) # Placeholder
])

 I tillegg vil jeg sjekke om modellen predikerer bedre hvis man for hver manglende data legger til en kolonne for hver variabel som tilsier om variablen manglet før imputasjonen. For å gjøre dette brukes MissingIndicator.

#### Ulike imputeringsstrategier

In [1970]:
imputers = [SimpleImputer(strategy="mean"), 
                  SimpleImputer(strategy="median"),
                  SimpleImputer(strategy="most_frequent"),
                  KNNImputer(n_neighbors=3),
                  KNNImputer(n_neighbors=5),
                  KNNImputer(n_neighbors=7)
                  ]

# Featureunion kombinere både imputerinsstrategi og indikatorer som lar dem kjøre parallelt
imputer_strats = imputers + [FeatureUnion(transformer_list=[(("imputer", imputer)), ("indicators", MissingIndicator(error_on_new=False))]) for imputer in imputers]

#### Ulike preprocessingsstrategier

Vil sjekke om alle stegene nevnt tidligere under preprocessing har noe effekt eller om det er bedre uten dem. Merk at minstekravet for å trene en regresjonsmodell er å bare ha numeriske variabler og ikke ha noe manglende data. Derfor er både OneHotEncoder og en imputerteringsstrategi i begge. I tillegg bruker jeg StandardScaler på begge da dette vil skalere datasettet og gjøre det lettere for regresjonsmodellen.

Siden vi tidligere har endret på categorical_cols og numeric_cols hvor vi la til ekstra og fjernet noen variabler, må man legge til og fjerne et par variabler for metoden uten ekstra preprocessing-steps.

In [1971]:
# OneHotEncoder + StandardScaler
minimum_preprocessing = Pipeline([ 
                          # Bruker OneHotEncoder på alle katergoriske variabler
                          # Skalerer og imputerer på numeriske variabler
                          ("encoder", ColumnTransformer([
                              ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))]), categorical_cols[:-4]), # Kategoriske variabler
                              ("num", Pipeline([("imputer", None), ("scaler", StandardScaler())]), numeric_cols + ['blodtrykk', 'respirasjonsfrekvens', 'lungefunksjon'])], # Numeriske variabler
                              remainder="passthrough") # Resten hoppes over
                           )
                          ])

# OneHotEncoder + StandardScaler + tidligere nevnt funksjoner for feature engineering
all_preprocessing = Pipeline([("transformer", FunctionTransformer(transform_X)), 
                          # Bruker OneHotEncoder på alle katergoriske variabler
                          # Skalerer og imputerer på numeriske variabler
                          ("encoder", ColumnTransformer([
                              ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))]), categorical_cols), # Kategoriske variabler
                              ("num", Pipeline([("imputer", None), ("scaler", StandardScaler())]), numeric_cols)], # Numeriske variabler
                              remainder="passthrough" # Resten hoppes over
                              ).set_output(transform="pandas")),
                          ("corr_drop", FunctionTransformer(drop_high_corr))]
                         )

preprocessors = [minimum_preprocessing, all_preprocessing]

Det som er spesielt med RandomizedCVSearch (og GridCVSearch), er at den har en innebygd kryssvalidering som deler datasettet opp i k-deler. Da trener den opp på k-1 deler og predikerer på siste del. Som standard er delt inn i fem k-deler. Det betyr at den i praksis deler datasettet 5 ulike kombinasjoner og scorer dem ut fra scorings-metode, i vårt tilfelle rmse. Resultatene lagres i cv_results_. En fordel da er at man kan slå sammen trenings og valideringsdata, da valideringsdata brukes til å finne hyperparametre, noe RandomizedCVSearch gjør for deg ut fra scoringsmetoden. Det er verdt å merke seg at det likevel ikke skjer noe datalekasje, nettopp på grunn av kryssvalideringsmetoden.

In [1972]:
X_train_val, y_train_val = pd.concat([X_train, X_val]), pd.concat([y_train, y_val])

### LinearRegression

In [1973]:
lr_params = [
    {
        "preprocessing": preprocessors,
        "preprocessing__encoder__num__imputer": imputer_strats,
        "model": [LinearRegression()],
        "model__fit_intercept": [True, False],  
        "model__positive": [True, False]
    }
    ]

print()
lr_search = RandomizedSearchCV(pipeline, lr_params, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42)
lr_search.fit(X_train_val, y_train_val)
print(f"Beste hyperparametere for LinearRegression: {lr_search.best_params_}")

/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


19 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/s

Beste hyperparametere for LinearRegression: {'preprocessing__encoder__num__imputer': KNNImputer(n_neighbors=7), 'preprocessing': Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sykdom_underkategori',
                                                   'kreft', 'kjønn',
                                                   'etnisitet', 'diabetes',
                                 

La oss se på hvordan cv_results_ ser ut i en dataframe.

In [1974]:
lr_results = pd.DataFrame(lr_search.cv_results_).sort_values(by="rank_test_score", ignore_index=True)
lr_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessing__encoder__num__imputer,param_preprocessing,param_model__positive,param_model__fit_intercept,param_model,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,17.418440,4.912590,2.121793,0.413506,KNNImputer(n_neighbors=7),"(ColumnTransformer(remainder='passthrough',\n ...",False,False,LinearRegression(),{'preprocessing__encoder__num__imputer': KNNIm...,-16.344651,-23.928902,-21.596732,-18.555625,-18.856861,-19.856554,2.632015,1
1,0.304298,0.065628,0.074143,0.052309,SimpleImputer(strategy='median'),"(ColumnTransformer(remainder='passthrough',\n ...",False,False,LinearRegression(),{'preprocessing__encoder__num__imputer': Simpl...,-16.330449,-23.957917,-21.580577,-18.570909,-18.862945,-19.860559,2.640716,2
2,0.492552,0.277205,0.079536,0.030611,"FeatureUnion(transformer_list=[('imputer',\n ...","(ColumnTransformer(remainder='passthrough',\n ...",False,False,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,-16.388596,-23.939885,-21.606436,-18.600217,-18.874408,-19.881909,2.619244,3
3,0.244635,0.062028,0.070507,0.041754,"FeatureUnion(transformer_list=[('imputer', Sim...","(ColumnTransformer(remainder='passthrough',\n ...",False,False,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,-16.397601,-23.934664,-21.606647,-18.596689,-18.898598,-19.886840,2.613753,4
4,6.796365,1.406700,2.745202,1.454735,"FeatureUnion(transformer_list=[('imputer', KNN...","(ColumnTransformer(remainder='passthrough',\n ...",False,True,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,-16.395348,-23.959819,-21.616600,-18.591621,-18.879345,-19.888546,2.625428,5
5,7.307114,1.273227,0.000000,0.000000,"FeatureUnion(transformer_list=[('imputer', KNN...",(FunctionTransformer(func=<function transform_...,False,False,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
6,10.900766,1.219854,0.000000,0.000000,"FeatureUnion(transformer_list=[('imputer', KNN...",(FunctionTransformer(func=<function transform_...,True,False,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
7,1.903002,0.950848,0.000000,0.000000,"FeatureUnion(transformer_list=[('imputer', Sim...",(FunctionTransformer(func=<function transform_...,False,True,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
8,0.216697,0.088681,0.024520,0.020991,SimpleImputer(),"(ColumnTransformer(remainder='passthrough',\n ...",True,True,LinearRegression(),{'preprocessing__encoder__num__imputer': Simpl...,-16.605402,NaN,-21.892742,-18.818286,NaN,NaN,NaN,6
9,6.187060,1.675514,0.640726,0.523436,"FeatureUnion(transformer_list=[('imputer', KNN...","(ColumnTransformer(remainder='passthrough',\n ...",True,True,LinearRegression(),{'preprocessing__encoder__num__imputer': Featu...,-16.622235,NaN,-21.905502,-18.827907,NaN,NaN,NaN,6


Merk at mean_test_score viser rmse, men den viser negativ verdi. Da tar vi absolutt verdien av dem, og finner beste validerings-rmse deretter.

In [1975]:
lr_results["mean_test_score"] = abs(lr_results["mean_test_score"])
lr_rmse = lr_results.loc[:, "mean_test_score"][0]
print(f"Beste rmse for LinearRegression: {lr_rmse}")

Beste rmse for LinearRegression: 19.856554250035565


Man kan også lett hente ut modellen som gjorde det best ved å bruke best_estimator_

In [1976]:
best_lr = lr_search.best_estimator_

Samme metode gjøres for de andre modellene.

### ElasticNet

In [1977]:
elastic_params = [
    {
        "preprocessing": preprocessors,
        "preprocessing__encoder__num__imputer": imputer_strats,
        "model": [ElasticNet(random_state=42)],
        "model__alpha": [0.001, 0.01, 0.1, 1, 10, 100],
        "model__max_iter": np.arange(1000, 5000, 1000),
        "model__l1_ratio": np.arange(0.0, 1.0, 0.1), 
        "model__tol": [0.0001, 0.00001, 0.001]
    }
    ]

print()
elastic_search = RandomizedSearchCV(pipeline, elastic_params, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42)
elastic_search.fit(X_train_val, y_train_val)
print(f"Beste hyperparametere for ElasticNet: {elastic_search.best_params_}")
elastic_results = pd.DataFrame(elastic_search.cv_results_).sort_values(by="rank_test_score", ignore_index=True)
elastic_results["mean_test_score"] = abs(elastic_results["mean_test_score"])
elastic_rmse = elastic_results.loc[:, "mean_test_score"][0]
print(f"Beste rmse for ElasticNet: {elastic_rmse}")
best_elastic = elastic_search.best_estimator_

/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/s

Beste hyperparametere for ElasticNet: {'preprocessing__encoder__num__imputer': FeatureUnion(transformer_list=[('imputer', KNNImputer()),
                               ('indicators',
                                MissingIndicator(error_on_new=False))]), 'preprocessing': Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sykdom_underkategori',
                                  

### RandomForestRegressor

In [1978]:
rf_params = [
    {
        "preprocessing": preprocessors,
        "preprocessing__encoder__num__imputer": imputer_strats,
        "model": [RandomForestRegressor(random_state=42)],
        "model__max_depth": [10, 20, None],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4, 10],
        "model__max_features": [1.0, "sqrt", "log2"],
        "model__bootstrap": [True, False]
    }
    ]

print()
rf_search = RandomizedSearchCV(pipeline, rf_params, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42)
rf_search.fit(X_train_val, y_train_val)
print(f"Beste hyperparametere for RandomForestRegressor: {rf_search.best_params_}")
rf_results = pd.DataFrame(rf_search.cv_results_).sort_values(by="rank_test_score", ignore_index=True)
rf_results["mean_test_score"] = abs(rf_results["mean_test_score"])
rf_rmse = rf_results.loc[:, "mean_test_score"][0]
print(f"Beste rmse for RandomForestRegressor: {rf_rmse}")
best_rf = rf_search.best_estimator_


/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/s

Beste hyperparametere for RandomForestRegressor: {'preprocessing__encoder__num__imputer': KNNImputer(n_neighbors=7), 'preprocessing': Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sykdom_underkategori',
                                                   'kreft', 'kjønn',
                                                   'etnisitet', 'diabetes',
                            

### GradientBoostingRegressor

In [1979]:
gb_params = [
    {
        "preprocessing": preprocessors,
        "preprocessing__encoder__num__imputer": imputer_strats,
        "model": [GradientBoostingRegressor(random_state=42)],
        "model__n_estimators": [50, 100, 200, 300],
        "model__learning_rate": [0.01, 0.05, 0.1, 0.2], 
        "model__max_depth": [3, 4, 5, 6],
        "model__min_samples_split": [2, 5, 10],  
        "model__min_samples_leaf": [1, 2, 4, 10],  
        "model__max_features": [None, "sqrt", "log2"]  
    }
    ]

print()
gb_search = RandomizedSearchCV(pipeline, rf_params, scoring="neg_root_mean_squared_error", n_jobs=-1, random_state=42)
gb_search.fit(X_train_val, y_train_val)
print(f"Beste hyperparametere for GradientBoostingRegressor: {gb_search.best_params_}")
gb_results = pd.DataFrame(gb_search.cv_results_).sort_values(by="rank_test_score", ignore_index=True)
gb_results["mean_test_score"] = abs(gb_results["mean_test_score"])
gb_rmse = gb_results.loc[:, "mean_test_score"][0]
print(f"Beste rmse for GradientBoostingRegressor: {gb_rmse}")
best_gb = gb_search.best_estimator_

/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sheldondyrdal/opt/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/s

Beste hyperparametere for GradientBoostingRegressor: {'preprocessing__encoder__num__imputer': KNNImputer(n_neighbors=7), 'preprocessing': Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sykdom_underkategori',
                                                   'kreft', 'kjønn',
                                                   'etnisitet', 'diabetes',
                        

### Beste model

In [2019]:
models_rmse = {
    baseline: rmse_baseline,
    best_lr: lr_rmse,
    best_elastic: elastic_rmse,
    best_rf: rf_rmse,
    best_gb: gb_rmse
    }

models_df = pd.DataFrame(list(models_rmse.items()), columns=["Modell", "rmse"])
models_df["Navn"] = "DummyRegressor", "LinearRegression", "ElasticNet", "RandomForestRegressor", "GradientBoostingRegressor"
models_df = models_df.sort_values(by="rmse", ignore_index=True).reindex(columns=["Navn", "Modell", "rmse"])
models_df

,Navn,Modell,rmse
0,RandomForestRegressor,"((ColumnTransformer(remainder='passthrough',\n...",19.584668
1,GradientBoostingRegressor,"((ColumnTransformer(remainder='passthrough',\n...",19.584668
2,LinearRegression,"((ColumnTransformer(remainder='passthrough',\n...",19.856554
3,ElasticNet,"((ColumnTransformer(remainder='passthrough',\n...",19.905218
4,DummyRegressor,DummyRegressor(),20.603791


In [2020]:
best_model = models_df["Modell"][0]
best_val_rmse = models_df["rmse"][0]
best_model_name = models_df["Navn"][0]
print(f"Beste modellen er {best_model_name} med rmse på trening og valideringsdata: {best_val_rmse}")
best_model

Beste modellen er RandomForestRegressor med rmse på trening og valideringsdata: 19.584668000533682


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('encoder',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('ohe',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   ['sykdom_underkategori',
                                                                    'kreft',
                                                                    'kjønn',
                                                                    'etnisitet',
                                                                    'diabetes',
                                                                    'demens']),
                                                                  ('num',
                                                                   Pipeline(...
                                                                    'koma_score',
                                                                    'fysiologisk_score',
                                                                    'apache_fysiologisk_score',
                                                                    'overlevelsesestimat_2mnd',
                                                                    'overlevelsesestimat_6mnd',
                                                                    'lege_overlevelsesestimat_2mnd',
                                                                    'lege_overlevelsesestimat_6mnd',
                                                                    'blodtrykk',
                                                                    'respirasjonsfrekvens',
                                                                    'lungefunksjon'])]))])),
                ('model',
                 RandomForestRegressor(max_depth=20, max_features='log2',
                                       min_samples_leaf=10, min_samples_split=5,
                                       random_state=42))])

#### Generaliseringsevne

In [2021]:
predictions = best_model.predict(X_test)
test_rmse = root_mean_squared_error(y_test, predictions)
test_rmse 

19.03240726478451

#### Visualisering av generaliseringsevnen

In [2029]:
df_results = pd.DataFrame({
    "Faktiske verdier": y_test,
    "Predikerte verdier": predictions
})

fig = make_subplots(rows=2, cols=1)

scatter1 = px.scatter(df_results, y=["Faktiske verdier", "Predikerte verdier"])

for trace in scatter1.data:
    fig.add_trace(trace, row=1, col=1)
    fig.update_yaxes(title_text="Oppholdslengde", row=1, col=1)

scatter2 = px.scatter(df_results, x="Faktiske verdier", y="Predikerte verdier", color_discrete_sequence=["purple"])

for trace in scatter2.data:
    fig.add_trace(trace, row=2, col=1)
    fig.update_xaxes(title_text="Faktiske verdier", row=2, col=1)
    fig.update_yaxes(title_text="Predikerte verdier", row=2, col=1)

fig.update_layout(title_text=f"Faktiske vs Predikerte verdier, test RMSE: {round(test_rmse, 2)}")
fig.write_image("images/generaliseringsevnen_til_best_modell.png")
fig.show()

## Implementering

Målet nå er å implementere den beste modellen til en nettside slik at brukeren kan legge inn relevante data for å predikere sykehusopphold. Dette vil da være data modellen ikke har sett før, slik at man kan bruke hele datasettet fra "raw data" til å trene opp modellen. Dette gjør man for at modellen skal få mest mulig data å trene på før man implenterer den til en nettside.

In [2030]:
best_model.fit(pd.concat([X_train_val, X_test]), pd.concat([y_train_val, y_test]))

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('encoder',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('ohe',
                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   ['sykdom_underkategori',
                                                                    'kreft',
                                                                    'kjønn',
                                                                    'etnisitet',
                                                                    'diabetes',
                                                                    'demens']),
                                                                  ('num',
                                                                   Pipeline(...
                                                                    'koma_score',
                                                                    'fysiologisk_score',
                                                                    'apache_fysiologisk_score',
                                                                    'overlevelsesestimat_2mnd',
                                                                    'overlevelsesestimat_6mnd',
                                                                    'lege_overlevelsesestimat_2mnd',
                                                                    'lege_overlevelsesestimat_6mnd',
                                                                    'blodtrykk',
                                                                    'respirasjonsfrekvens',
                                                                    'lungefunksjon'])]))])),
                ('model',
                 RandomForestRegressor(max_depth=20, max_features='log2',
                                       min_samples_leaf=10, min_samples_split=5,
                                       random_state=42))])

Lagrer beste modell ved hjelp av pickle

In [2031]:
pickle.dump(best_model, open('model.pkl', 'wb'))

## Predikere på nytt datasett

Importerer nye datasett fra "sample data" for å se hvordan modellen predikerer ny data. Resultatet eksporteres til en csv-fil.

In [2032]:
df_demographic = pd.read_csv("sample_data/demographic.csv")
df_hospital = pd.read_csv("sample_data/hospital.csv")
df_physiological = pd.read_csv("sample_data/physiological.txt", sep="\t")
df_severity = pd.read_json("sample_data/severity.json")

X_sample = prepare_data(df_hospital, df_demographic, df_physiological, df_severity)
X_sample = remove_cols(X_sample)

In [2033]:
predictions_df = pd.DataFrame({"pasient_id": X_sample["pasient_id"], "predikert_sykehusopphold": np.round(best_model.predict(X_sample))})
assert not (predictions_df["predikert_sykehusopphold"] < 0).any(), "Det predikeres negative verdier"
predictions_df

,pasient_id,predikert_sykehusopphold
0,1,12.0
1,22,15.0
2,25,15.0
3,26,9.0
4,29,21.0
...,...,...
1360,9049,15.0
1361,9050,9.0
1362,9078,20.0
1363,9080,8.0


In [2047]:
fig = px.histogram(predictions_df, x="predikert_sykehusopphold", histnorm="probability density")
dmax = 0.09
fig.add_trace(go.Scatter(x=[np.mean(predictions_df["predikert_sykehusopphold"]), np.mean(predictions_df["predikert_sykehusopphold"])], y=[0, dmax], mode="lines", line=dict(color="yellow"),name="Middelverdi"))
fig.add_trace(go.Scatter(x=[np.median(predictions_df["predikert_sykehusopphold"]), np.median(predictions_df["predikert_sykehusopphold"])], y=[0, dmax], mode="lines", line=dict(color="purple"),name="Median"))
fig.add_trace(go.Scatter(x=[stats.mode(predictions_df["predikert_sykehusopphold"])[0], stats.mode(predictions_df["predikert_sykehusopphold"])[0]], y=[0, dmax], mode="lines", line=dict(color="orange"),name="Modus"))
fig.write_image(f"images/sample_data_pred.png")
fig.show()

In [2048]:
predictions_df.to_csv("predictions.csv", index=False, encoding="utf-8")